### Install Libraries

In [5]:
from pathlib import Path
import pandas as pd
from ortools.sat.python import cp_model
import pandas as pd
import numpy as np
import random
import os
from ortools.sat.python.cp_model import LinearExpr

os.chdir("/home/ashaar/medical-rotation-scheduling")
# os.chdir("/Users/ashaar/medical-rotation-scheduling")


In [ ]:
#####
# GRAD_REQ = {
#     "R1": {
#         ("Medical Teams",): (5, 6, 7, 8),
#         ("AMAU",): (1, 2),
#         ("Cardiology",): (2,),
#         ("Infectious Disease",): (1, 2),
#         ("Endocrine",): (1, 2),
#     },
#     "R2": {
#         ("Senior Rotation",): (1,),
#         ("CCU",): (2,),
#         ("MICU",): (2,),
#         ("Nephrology",): (1, 2),
#         ("Neurology",): (1,),
#         ("Cardiology",): (1,),
#         ("Geriatrics",): (1,),
#         ("AMAU",): (1, 2),
#         ("Al Khor",): (1,),
#         ("MOP",): (1,),
#     },
#     "R3": {
#         ("Senior Rotation",): (1, 2),
#         ("Oncology",): (1,),
#         ("Hematology",): (1,),
#         ("Al Wakra",): (1, 2),
#         ("GI",): (2,),
#         ("Pulmonology",): (2,),
#         ("Rheumatology",): (1,),
#         ("MOP",): (1,),
#         ("AMAU",): (1,),
#         ("Cardiology", "ED", "Medical Consultation"): (1,),
#     },
#     "R4": {
#         ("Registrar Rotation",): (4, 5, 6),
#         ("Medical Consultation",): (4, 5, 6),
#         ("Al Wakra",): (0, 1, 2),
#         ("Al Khor",): (0, 1),
#         ("Hematology", "Oncology"): (0, 1),
#     },
#     }

### Global Variables

In [6]:
# -------------------------------
# Global Constants and Parameters
# -------------------------------
INPUT_FILE = "real_example_input.xlsx"
OUTPUT_XLSX = "output_file.xlsx"
BLOCKS = 13

# -------------------------------
# Rotation Definitions
# -------------------------------
ROTATIONS = [
    "Cardiology", "Endocrine", "Infectious Disease", "AMAU", "Nephrology",
    "Neurology", "CCU", "MICU", "Al Khor", "MOP", "Geriatrics", "Hematology",
    "Oncology", "Al Wakra", "GI", "Pulmonology", "Rheumatology", "ED",
    "Medical Consultation", "Medical Teams", "Senior Rotation",
    "Registrar Rotation",
]

LEAVE_ROTATION = "LEAVE"
ROTATIONS.append(LEAVE_ROTATION)

TRANSFER_ROTATION = "TRANSFER"
ROTATIONS.append(TRANSFER_ROTATION)

# -------------------------------
# Leave-Eligible Rotations by PGY
# -------------------------------
LEAVE_ALLOWED = {
    "R1": {"Endocrine", "AMAU", "Infectious Disease"},
    "R2": {"MOP", "Nephrology", "AMAU"},
    "R3": {"GI", "Pulmonology", "AMAU"},
    "R4": {"Medical Consultation"},
    "R_NEURO": {"AMAU",}
}

# -------------------------------
# Graduation Requirements by PGY
# -------------------------------
#TODO: Calculate the average and then get negative score for larger variance. (Medical Teams)
GRAD_REQ = {
    "R1": {
        ("Medical Teams",): (4, 5, 6, 7), #TODO: ADD LOCKED.
        ("AMAU",): (1, 2),
        ("Cardiology",): (2,),
        ("Infectious Disease",): (1, 2),
        ("Endocrine",): (1, 2),
    },
    "R2": {
        ("Senior Rotation",): (1,2),
        ("CCU",): (2,),
        ("MICU",): (2,),
        ("Nephrology",): (1, 2),
        ("Neurology",): (1,),
        ("Cardiology",): (1,),
        ("Geriatrics",): (1,),
        ("AMAU",): (1, 2),
        ("Al Khor",): (0, 1),
        ("MOP",): (1,),
    },
    "R3": {
        ("Senior Rotation",): (2,),
        ("Oncology",): (1,),
        ("Hematology",): (1,),
        ("Al Wakra",): (1,),
        ("GI",): (2,),
        ("Pulmonology",): (2,),
        ("Rheumatology",): (1,),
        ("MOP",): (1,),
        ("AMAU",): (1,),
        ("Cardiology", "ED", "Medical Consultation"): (1,),
    },
    "R4": {
        ("Registrar Rotation",): (5, 6),
        ("Medical Consultation",): (3, 4, 5,),
        ("Al Wakra",): (1, 2),
        ("Al Khor",): (1, 2),
        ("Hematology", "Oncology"): (1, 2),
    },
    "R4_Chiefs": {
        ("Registrar Rotation", ): (5, 6, 7),
        ("Medical Consultation",): (6, 7, 8),
    },
    "R_NEURO": {
        ("Medical Teams", ): (3, ),
        ("AMAU",): (3,4,),
        ("MICU",): (1,),
        ("Rheumatology",): (1,),
        ("ED",): (2,),
        ("TRANSFER",): (2,)
    },
}

# -------------------------------
# Per-block Minimum Requirements
# -------------------------------
PER_BLOCK_MIN = {
    "Cardiology": 11,
    "AMAU": 11,
    "CCU": 7,
    "MICU": 7,
    "Al Khor": 6,
    "MOP": 6,
    "Hematology": 5,
    "Oncology": 5,
    "Al Wakra": 9,
    "Medical Consultation": 10,
    "Medical Teams": 20,
    # "Registrar Rotation": 20,
    # "Senior Rotation": 10,
}


# -------------------------------
# Eligibility Matrix
# -------------------------------
ELIGIBILITY = {
    pgy: {rot for group in GRAD_REQ[pgy] for rot in group}
    for pgy in GRAD_REQ
}

for pgy in ELIGIBILITY:
    ELIGIBILITY[pgy].add(LEAVE_ROTATION)

# -------------------------------
# Rotation Group Definitions
# -------------------------------
group_defs = {
    "Floater": {"Nephrology", "Endocrine"},
    "2ndOnCall": {"GI", "Rheumatology", "Pulmonology"},
}


In [7]:
ELIGIBILITY

{'R1': {'AMAU',
  'Cardiology',
  'Endocrine',
  'Infectious Disease',
  'LEAVE',
  'Medical Teams'},
 'R2': {'AMAU',
  'Al Khor',
  'CCU',
  'Cardiology',
  'Geriatrics',
  'LEAVE',
  'MICU',
  'MOP',
  'Nephrology',
  'Neurology',
  'Senior Rotation'},
 'R3': {'AMAU',
  'Al Wakra',
  'Cardiology',
  'ED',
  'GI',
  'Hematology',
  'LEAVE',
  'MOP',
  'Medical Consultation',
  'Oncology',
  'Pulmonology',
  'Rheumatology',
  'Senior Rotation'},
 'R4': {'Al Khor',
  'Al Wakra',
  'Hematology',
  'LEAVE',
  'Medical Consultation',
  'Oncology',
  'Registrar Rotation'},
 'R4_Chiefs': {'LEAVE', 'Medical Consultation', 'Registrar Rotation'},
 'R_NEURO': {'AMAU',
  'ED',
  'LEAVE',
  'MICU',
  'Medical Teams',
  'Rheumatology',
  'TRANSFER'}}

### Parser

In [8]:
# -------------------------------
# Excel Input Parser and Index Maps
# -------------------------------
def parse_input(path: str):
    """Read Excel, return residents, PGYs, leave dict, forced/forbidden maps."""
    df = pd.read_excel(path).fillna({
        "Leave1Block": 0,
        "Leave2Block": 0,
        "Leave1Half": "",
        "Leave2Half": ""
    })

    residents = df["ID"].tolist()
    pgys = df["PGY"].tolist()
    leave_dict = {}
    forced_assignments = {}
    forbidden_assignments = {}

    for row in df.itertuples(index=False):
        b1, b2 = int(row.Leave1Block), int(row.Leave2Block)
        full, half = set(), set()

        if b1 and b1 == b2:
            full.add(b1)
        else:
            if b1:
                half.add(b1)
            if b2:
                half.add(b2)

        leave_dict[row.ID] = {
            "pgy": row.PGY,
            "full": full,
            "half": half
        }

        for b in range(1, 14):
            val = getattr(row, f"Block_{b}", "")
            val = str(val).strip()
            if not val or val.lower() in {"nan", "none"}:
                continue
            if val.startswith("!"):
                forbidden_assignments[(row.ID, b - 1)] = val[1:]
            else:
                forced_assignments[(row.ID, b - 1)] = val

    return residents, pgys, leave_dict, forced_assignments, forbidden_assignments

residents, pgys, leave_dict, forced_assignments, forbidden_assignments = (
    parse_input(os.path.join("sample_data", INPUT_FILE))
)

n = len(residents)

rotation_to_idx = {rot: i for i, rot in enumerate(ROTATIONS)}
idx_to_rotation = {i: rot for rot, i in rotation_to_idx.items()}
LEAVE_IDX = rotation_to_idx[LEAVE_ROTATION]

rotation_to_idx

forced_assignments

{}

In [9]:
set(rotation_to_idx.keys()) == set(ROTATIONS)


True

### Decision Variable Definition and Assignment Domain


In [10]:
# -------------------------
# CP-SAT Model Setup
# -------------------------
model = cp_model.CpModel()

# x[r, b]: Integer variable representing the rotation index assigned to
# resident r in block b (0-based: 0..12).
# - If block b is a full leave block, x[r, b] is fixed to LEAVE_IDX.
# - Else, x[r, b] is constrained to eligible rotations based on PGY and
#   further restricted to LEAVE_ALLOWED if it's a half-leave block.
x = {}
for r in range(n):
	res_id = residents[r]
	pgy = pgys[r]
	for b in range(BLOCKS):
		if (b + 1) in leave_dict[res_id]["full"]:
			x[r, b] = model.NewIntVarFromDomain(
				cp_model.Domain.FromValues([LEAVE_IDX]), f"x_{r}_{b}"
			)
		else:
			eligible = [
				rotation_to_idx[rot]
				for rot in ELIGIBILITY[pgy]
				if rot in rotation_to_idx and
				rotation_to_idx[rot] != LEAVE_IDX
			]
			if (b + 1) in leave_dict[res_id]["half"]:
				eligible = [
					rotation_to_idx[rot]
					for rot in ELIGIBILITY[pgy]
					if rot in rotation_to_idx and
					rot in LEAVE_ALLOWED.get(pgy, set()) and
					rotation_to_idx[rot] != LEAVE_IDX
				]
			x[r, b] = model.NewIntVarFromDomain(
				cp_model.Domain.FromValues(eligible), f"x_{r}_{b}"
			)


# -------------------------
# Indicator Variables y[r, b, rot]
# -------------------------
# For each rotation (including LEAVE), y[r, b, rot] = 1 iff resident r
# is assigned to rotation `rot` in block b. These are boolean indicators
# tied to the integer decision variable x[r, b].
y = {}
for r in range(n):
	for b in range(BLOCKS):  # BLOCKS = 13, blocks 0 to 12
		for rot in ROTATIONS:
			rot_id = rotation_to_idx[rot]
			var = model.NewBoolVar(f"y_{r}_{b}_{rot}")
			model.Add(x[r, b] == rot_id).OnlyEnforceIf(var)
			model.Add(x[r, b] != rot_id).OnlyEnforceIf(var.Not())
			y[r, b, rot] = var

# Enforce that each block has exactly one assigned rotation
for r in range(n):
	for b in range(BLOCKS):
		model.AddExactlyOne([y[r, b, rot] for rot in ROTATIONS])


# -------------------------
# Leave Weights
# -------------------------
# Weight each resident-block pair by:
#   - 1 if it's a half-leave block (less time available),
#   - 2 otherwise.
# These weights are used to scale contributions to minimum coverage constraints.
half_leave_weights = {
	(r, b): 1 if (b + 1) in leave_dict[residents[r]]["half"] else 2
	for r in range(n)
	for b in range(BLOCKS)
}

# -------------------------
# R_NEURO Hard Constraints
# -------------------------
for r in range(n):
    if pgys[r] == "R_NEURO":
        # First block must be Medical Teams
        model.Add(x[r, 0] == rotation_to_idx["Medical Teams"])
        model.Add(x[r, 1] == rotation_to_idx["Medical Teams"])
        model.Add(x[r, 2] == rotation_to_idx["Medical Teams"])
        
        
        # Last two blocks must be TRANSFER
        model.Add(x[r, 11] == rotation_to_idx["TRANSFER"])
        model.Add(x[r, 12] == rotation_to_idx["TRANSFER"])

# -------------------------
# Per-Block Minimum Rotation Coverage
# -------------------------
# For each rotation `rot` that has a minimum coverage requirement:
# In every block, the sum of weighted assignments to `rot`
# must be ≥ 2 × PER_BLOCK_MIN[rot].
# for b in range(BLOCKS):
# 	for rot in PER_BLOCK_MIN:
# 		model.Add(
# 			sum(half_leave_weights[r, b] * y[r, b, rot] for r in range(n))
# 			>= 2 * PER_BLOCK_MIN[rot]
# 		)

# -------------------------
# Graduation Requirements
# -------------------------
# Each resident must fulfill graduation requirements for specific rotation
# groups, unless exempt (R3s with full leave can skip one group).

for r in range(n):
	res_id = residents[r]
	pgy = pgys[r]
	leave_blocks = leave_dict[res_id]["full"]

	for rotation_group, required_counts in GRAD_REQ[pgy].items():
		if (
			pgy == "R3"
			and set(rotation_group) == {"Cardiology", "ED", "Medical Consultation"}
			and leave_blocks
		):
			continue

		count_vars = []
		for b in range(BLOCKS):
			if (b+1) not in leave_blocks:
				for rot in rotation_group:
					if rot in rotation_to_idx:
						count_vars.append(y[r, b, rot])

		if not count_vars:
			print(f"[GRAD EMPTY] {res_id} ({pgy}) → {rotation_group} → SKIPPED")
		elif len(count_vars) < min(required_counts):
			print(f"[GRAD TOO FEW BLOCKS] {res_id} ({pgy}) needs ≥{min(required_counts)} "
				f"but only has {len(count_vars)} possible")

		model.Add(sum(count_vars) >= min(required_counts))
		model.Add(sum(count_vars) <= max(required_counts))



# -------------------------
# On-call Rules and Coverage
# -------------------------
# 1. Every block must have exactly 10 Senior Rotations.
# 2. Every block must have exactly 20 Registrar Rotations.
# 3. For 2nd-on-call rotations (GI, Rheumatology, Pulmonology), the total
#    weighted count (3 if half leave, 6 otherwise) must be ≥ 60.
# 4. For Floater rotations (Nephrology, Endocrine), ensure at least 10
#    residents assigned in each block.

second_wt = {
	(r, b): 3 if (b + 1) in leave_dict[residents[r]]["half"] else 6
	for r in range(n)
	for b in range(BLOCKS)
}

print(f"Residents: {n}")
for b in range(BLOCKS):
	n_leave = sum(1 for r in range(n) if (b + 1) in leave_dict[residents[r]]["full"])
	print(f"Block {b+1}: {n_leave} on full leave")

for b in range(BLOCKS):
	# Enforce fixed counts
	model.Add(sum(y[r, b, "Senior Rotation"] for r in range(n)) == 10)
	# model.Add(sum(y[r, b, "Senior Rotation"] for r in range(n)) <= 13)
 
	model.Add(sum(y[r, b, "Registrar Rotation"] for r in range(n)) == 20)

	# PER_BLOCK_MIN: simple unweighted counts
	for rot in PER_BLOCK_MIN:
		model.Add(
			sum(y[r, b, rot] for r in range(n)) >= PER_BLOCK_MIN[rot]
		)
  
	# Weighted 2nd on-call minimum
	# TODO: Add soft 2x +ve constraint where the sum is at least 64.
	model.Add(
		sum(
			second_wt[r, b] * y[r, b, rot]
			for r in range(n)
			if pgys[r] != "R_NEURO"
			for rot in group_defs["2ndOnCall"]
		) >= 60
	)

	# Floater (Nephrology + Endocrine) ≥ 10 residents
	model.Add(
		sum(
			y[r, b, rot]
			for r in range(n)
			if pgys[r] != "R_NEURO"
			for rot in group_defs["Floater"]
		) >= 10
	)
	
		# Extra Nephrology + Extra Endocrine = total - 5 each
	total_endo = sum(y[r, b, "Endocrine"] for r in range(n))
	total_neph = sum(y[r, b, "Nephrology"] for r in range(n))

	# Define new IntVar for extra_endo and extra_neph
	extra_endo = model.NewIntVar(0, n, f"extra_endo_b{b}")
	extra_neph = model.NewIntVar(0, n, f"extra_neph_b{b}")

	# Symbolic max(0, total - 5)
	model.AddMaxEquality(extra_endo, [0, total_endo - 5])
	model.AddMaxEquality(extra_neph, [0, total_neph - 5])
	
 	# R2s in MOP
	r2_in_mop = sum(
		y[r, b, "MOP"]
		for r in range(n)
		if pgys[r] == "R2"
	)

	# Neurology
	neuro = sum(y[r, b, "Neurology"] for r in range(n))

	# Combined condition
	# model.Add(neuro + r2_in_mop + extra_endo + extra_neph >= 4)

# -------------------------
# Post-Block-3 Medical Teams Coverage
# -------------------------
# From Block 4 onwards (index 3), enforce exactly 20 residents on
# Medical Teams per block.

model.Add(sum(y[r, 1, "Medical Teams"] for r in range(n)) >= 25)

for b in range(3, BLOCKS):
	model.Add(sum(y[r, b, "Medical Teams"] for r in range(n)) == 20)

# -------------------------
# Block-1 PGY Restrictions
# -------------------------
# PGY-1 residents must start in Medical Teams in Block 1.
# PGY-2 residents cannot be assigned to Senior Rotation in Block 1.

med_teams_idx = rotation_to_idx["Medical Teams"]
senior_idx = rotation_to_idx["Senior Rotation"]

for r in range(n):
	if pgys[r] == "R1":
		model.Add(x[r, 0] == med_teams_idx)
	elif pgys[r] == "R2":
		model.Add(x[r, 0] != senior_idx)

# -------------------------
# R1: No 6 Consecutive Medical Teams
# -------------------------
# For each PGY-1 resident, in any 6-block window, they cannot have all
# 6 blocks assigned to Medical Teams.

for r in range(n):
	if pgys[r] == "R1":
		for start in range(BLOCKS - 5):
			model.Add(
				sum(y[r, b, "Medical Teams"] for b in range(start, start + 6))
				<= 5
			)
	if pgys[r] == "R3" or pgys[r] == "R2":
		for b in range(BLOCKS - 1):
			model.AddBoolOr([
				y[r, b, "Senior Rotation"].Not(),
				y[r, b + 1, "Senior Rotation"].Not()
			])


# -------------------------
# Soft Objective Components
# -------------------------

# R2/R3 mix in Cardiology and AMAU
rotation_mix_score = []
for b in range(BLOCKS):
	for rot in ["Cardiology", "AMAU"]:
		eligible_vars = [
			y[r, b, rot]
			for r in range(n)
			if pgys[r] in ("R2", "R3")
		]
		if eligible_vars:
			mix_ok = model.NewBoolVar(f"mix_r2r3_b{b}_{rot}")
			model.Add(sum(eligible_vars) >= 2).OnlyEnforceIf(mix_ok)
			model.Add(sum(eligible_vars) < 2).OnlyEnforceIf(mix_ok.Not())
			rotation_mix_score.append(mix_ok)

# R1 penalties
penalty_r1_long_medical = []
penalty_r1_consec_cardiology = []

for r in range(n):
	if pgys[r] == "R1":
		for start in range(BLOCKS - 3):
			window = [y[r, b, "Medical Teams"] for b in range(start, start + 4)]
			all_four = model.NewBoolVar(f"r1_medical_4consec_{r}_{start}")
			model.Add(sum(window) == 4).OnlyEnforceIf(all_four)
			model.Add(sum(window) != 4).OnlyEnforceIf(all_four.Not())
			penalty_r1_long_medical.append(all_four)

		for b in range(BLOCKS - 1):
			both = model.NewBoolVar(f"r1_cardio_consec_{r}_{b}")
			model.AddBoolAnd([
				y[r, b, "Cardiology"],
				y[r, b + 1, "Cardiology"]
			]).OnlyEnforceIf(both)
			model.AddBoolOr([
				y[r, b, "Cardiology"].Not(),
				y[r, b + 1, "Cardiology"].Not()
			]).OnlyEnforceIf(both.Not())
			penalty_r1_consec_cardiology.append(both)

# R2 rewards
reward_r2_consec_micu = []
reward_r2_consec_ccu = []

for r in range(n):
	if pgys[r] == "R2":
		for b in range(BLOCKS - 1):
			both_micu = model.NewBoolVar(f"r2_micu_consec_{r}_{b}")
			model.AddBoolAnd([
				y[r, b, "MICU"],
				y[r, b + 1, "MICU"]
			]).OnlyEnforceIf(both_micu)
			model.AddBoolOr([
				y[r, b, "MICU"].Not(),
				y[r, b + 1, "MICU"].Not()
			]).OnlyEnforceIf(both_micu.Not())
			reward_r2_consec_micu.append(both_micu)


			both_ccu = model.NewBoolVar(f"r2_ccu_consec_{r}_{b}")
			model.AddBoolAnd([
				y[r, b, "CCU"],
				y[r, b + 1, "CCU"]
			]).OnlyEnforceIf(both_ccu)
			model.AddBoolOr([
				y[r, b, "CCU"].Not(),
				y[r, b + 1, "CCU"].Not()
			]).OnlyEnforceIf(both_ccu.Not())
			reward_r2_consec_ccu.append(both_ccu)
   
# R3 penalties: insufficient spacing between Senior blocks
penalty_r3_senior_spacing = []

for r in range(n):
	if pgys[r] == "R3":
		for b in range(BLOCKS - 1):
			p = model.NewBoolVar(f"r3_senior_consec_{r}_{b}")
			model.AddBoolAnd([
				y[r, b, "Senior Rotation"],
				y[r, b + 1, "Senior Rotation"]
			]).OnlyEnforceIf(p)
			model.AddBoolOr([
				y[r, b, "Senior Rotation"].Not(),
				y[r, b + 1, "Senior Rotation"].Not()
			]).OnlyEnforceIf(p.Not())
			penalty_r3_senior_spacing.append(p)

		for b in range(BLOCKS - 2):
			p = model.NewBoolVar(f"r3_senior_gap1_{r}_{b}")
			model.AddBoolAnd([
				y[r, b, "Senior Rotation"],
				y[r, b + 2, "Senior Rotation"]
			]).OnlyEnforceIf(p)
			model.AddBoolOr([
				y[r, b, "Senior Rotation"].Not(),
				y[r, b + 2, "Senior Rotation"].Not()
			]).OnlyEnforceIf(p.Not())
			penalty_r3_senior_spacing.append(p)

# R4 penalties: >5 consecutive Registrar rotations
penalty_r4_long_registrar = []

for r in range(n):
	if pgys[r] == "R4":
		for start in range(BLOCKS - 5):
			window = [y[r, b, "Registrar Rotation"] for b in range(start, start + 6)]
			too_long = model.NewBoolVar(f"r4_registrar_6consec_{r}_{start}")
			model.Add(sum(window) == 6).OnlyEnforceIf(too_long)
			model.Add(sum(window) != 6).OnlyEnforceIf(too_long.Not())
			penalty_r4_long_registrar.append(too_long)

# -------------------------
# Final Objective
# -------------------------
#TODO: Get preferences score (normalized to 1). Provide summary with the score.
model.Maximize(
	sum(rotation_mix_score)
	- sum(penalty_r1_long_medical)
	- sum(penalty_r1_long_medical)
	- sum(penalty_r1_consec_cardiology)
 
	+ sum(reward_r2_consec_micu)
	+ sum(reward_r2_consec_micu)
	+ sum(reward_r2_consec_ccu)
	+ sum(reward_r2_consec_ccu)
 
	- sum(penalty_r3_senior_spacing)
	- sum(penalty_r3_senior_spacing)
 
	- sum(penalty_r4_long_registrar)
	- sum(penalty_r4_long_registrar)
 
)


Residents: 183
Block 1: 0 on full leave
Block 2: 0 on full leave
Block 3: 0 on full leave
Block 4: 1 on full leave
Block 5: 1 on full leave
Block 6: 0 on full leave
Block 7: 0 on full leave
Block 8: 2 on full leave
Block 9: 1 on full leave
Block 10: 0 on full leave
Block 11: 0 on full leave
Block 12: 0 on full leave
Block 13: 0 on full leave


### Solving the Model and Writing the Schedule Output

In [11]:
# -------------------------
# Solve and Output Schedule
# -------------------------
solver = cp_model.CpSolver()
# solver.parameters.log_search_progress = True
# solver.parameters.cp_model_probing_level = 0
# solver.parameters.enumerate_all_solutions = False

status = solver.Solve(model)


if status not in (cp_model.FEASIBLE, cp_model.OPTIMAL):
    raise RuntimeError("No feasible solution found.")

output = []
for r in range(n):
    row = [residents[r]]
    for b in range(BLOCKS):
        rot = idx_to_rotation[solver.Value(x[r, b])]
        row.append(rot)
    output.append(row)


columns = ["Resident"] + [f"Block_{i+1}" for i in range(BLOCKS)]
output_df = pd.DataFrame(output, columns=columns)
output_df.to_excel(OUTPUT_XLSX, index=False)
output_df.head()


,Resident,Block_1,Block_2,Block_3,Block_4,Block_5,Block_6,Block_7,Block_8,Block_9,Block_10,Block_11,Block_12,Block_13
0,R1_001,Medical Teams,Medical Teams,AMAU,Infectious Disease,Medical Teams,Medical Teams,Cardiology,Medical Teams,AMAU,Endocrine,Medical Teams,Cardiology,Medical Teams
1,R1_002,Medical Teams,Medical Teams,AMAU,Cardiology,AMAU,Medical Teams,Endocrine,Infectious Disease,Medical Teams,Cardiology,Medical Teams,Infectious Disease,Medical Teams
2,R1_003,Medical Teams,Cardiology,Infectious Disease,Medical Teams,Medical Teams,Endocrine,Medical Teams,Infectious Disease,Medical Teams,AMAU,Cardiology,Medical Teams,AMAU
3,R1_004,Medical Teams,Medical Teams,AMAU,Medical Teams,Medical Teams,Cardiology,Infectious Disease,Medical Teams,Endocrine,Endocrine,Medical Teams,Medical Teams,Cardiology
4,R1_005,Medical Teams,Cardiology,Medical Teams,Infectious Disease,Medical Teams,Endocrine,AMAU,Endocrine,AMAU,Infectious Disease,Cardiology,Medical Teams,Medical Teams


In [ ]:
from collections import Counter
print("Forced assignments per block:")
for (rid, b), rot in forced_assignments.items():
    print(f"{rid} → Block {b+1}: {rot}")


### Rotation Distribution Summary by Block

In [ ]:
# -------------------------
# Rotation Distribution Summary by Block
# -------------------------
blocks = [f"Block_{i}" for i in range(1, 14)]

melted = output_df.melt(
    id_vars=["Resident"],
    value_vars=blocks,
    var_name="Block",
    value_name="Rotation"
)

count_df = (
    melted.groupby(["Rotation", "Block"])
    .size()
    .unstack(fill_value=0)
    .sort_index()
)

count_df.index.name = None
count_df.columns.name = None
count_df = count_df[blocks]
count_df.to_excel("rotation_distribution.xlsx")
count_df

### Draft (Extra):

In [ ]:
# TODO count in extra nephrology and endocrine.
model.Add(sum(y[r, b, "Neurology"] + (y[r, b, "Nephrology"] + y[r, b, "Endocrine "] - 10) for r in range(b)

In [ ]:
import pandas as pd

# ---------- helper ----------
def weight(r, b, leave_dict):
	"""2 full, 1 half"""
	return 1 if (b + 1) in leave_dict[r]["half"] else 2

# ---------- graduation range ----------
def check_grad(output_df, pgys, leave_dict):
	"""return {(resident,group):count} out-of-range"""
	viol = {}
	for r, row in output_df.iterrows():
		res = row["Resident"]
		pgy = pgys[r]
		full_leave = leave_dict[res]["full"]
		for grp, rng in GRAD_REQ[pgy].items():
			if pgy == "R3" and grp == (
				"Cardiology", "ED", "Medical Consultation"
			) and full_leave:
				continue
			ct = sum(row[f"Block_{b}"] in grp for b in range(1, BLOCKS + 1))
			if ct < min(rng) or ct > max(rng):
				viol[(res, grp)] = ct
	return viol

# ---------- staffing minima ----------
def check_block_min(output_df, leave_dict):
	"""return {(block,rot):count} below minimum"""
	viol = {}
	for b in range(1, BLOCKS + 1):
		for rot, req in PER_BLOCK_MIN.items():
			ct = 0
			for r, row in output_df.iterrows():
				if row[f"Block_{b}"] == rot:
					ct += weight(row["Resident"], b - 1, leave_dict)
			if ct < 2 * req:
				viol[(b, rot)] = ct
	return viol

# ---------- exact tallies ----------
def check_exact(output_df, leave_dict):
	"""return {(block,tag):count} wrong"""
	viol = {}
	for b in range(1, BLOCKS + 1):
		def tall(rot):
			return sum(
				weight(row["Resident"], b - 1, leave_dict)
				for _, row in output_df.iterrows()
				if row[f"Block_{b}"] == rot
			)
		if tall("Senior Rotation") != 20:
			viol[(b, "Senior")] = tall("Senior Rotation")
		if tall("Registrar Rotation") != 40:
			viol[(b, "Registrar")] = tall("Registrar Rotation")
		if b >= 4 and tall("Medical Teams") != 40:
			viol[(b, "MedTeams")] = tall("Medical Teams")
	return viol

# ---------- on-call groups ----------
def check_groups(output_df, leave_dict):
	"""return {(block,group):count} below target"""
	targets = {"FLOATER": 8, "NEURO": 4, "2NDONCALL": 20}
	viol = {}
	for b in range(1, BLOCKS + 1):
		for g, members in group_defs.items():
			ct = 0
			for _, row in output_df.iterrows():
				if row[f"Block_{b}"] in members:
					w = weight(row["Resident"], b - 1, leave_dict)
					ct += w if g == "2NDONCALL" else 1
			if ct < targets[g] * (2 if g == "2NDONCALL" else 1):
				viol[(b, g)] = ct
	return viol

# ---------- PGY first block ----------
def check_first(output_df, pgys):
	"""return list of resident ids violating first-block rule"""
	bad = []
	for r, row in output_df.iterrows():
		if pgys[r] == "R1" and row["Block_1"] != "Medical Teams":
			bad.append(row["Resident"])
		if pgys[r] == "R2" and row["Block_1"] == "Senior Rotation":
			bad.append(row["Resident"])
	return bad

# ---------- consecutive Med-Teams ----------
def check_consecutive(output_df, pgys):
	"""return list of resident ids with >5 consecutive MedTeams"""
	bad = []
	for r, row in output_df.iterrows():
		if pgys[r] != "R1":
			continue
		seq = 0
		for b in range(4, BLOCKS + 1):
			if row[f"Block_{b}"] == "Medical Teams":
				seq += 1
				if seq > 5:
					bad.append(row["Resident"])
					break
			else:
				seq = 0
	return bad

# ---------- master validator ----------
def validate(output_df, residents, pgys, leave_dict):
	"""run all checks, return dict of violations"""
	return {
		"grad": check_grad(output_df, pgys, leave_dict),
		"block_min": check_block_min(output_df, leave_dict),
		"exact": check_exact(output_df, leave_dict),
		"groups": check_groups(output_df, leave_dict),
		"first_block": check_first(output_df, pgys),
		"consecutive": check_consecutive(output_df, pgys),
	}

# example:
viol = validate(output_df, residents, pgys, leave_dict)
print(viol)


In [ ]:
# -------------------------
# CP-SAT Model
# -------------------------
model = cp_model.CpModel()

x = {}
for r in range(n):
	res_id = residents[r]
	pgy = pgys[r]
	for b in range(BLOCKS):
		block = b + 1
		if block in leave_dict[res_id]["full"]:
			x[r, b] = model.NewIntVarFromDomain(
				cp_model.Domain.FromValues([LEAVE_IDX]), f'x_{r}_{b}'
			)
		else:
			eligible = [
				rotation_to_idx[rot]
				for rot in ELIGIBILITY[pgy]
				if rot in rotation_to_idx
			]
			if block in leave_dict[res_id]["half"]:
				eligible = [
					rotation_to_idx[rot]
					for rot in ELIGIBILITY[pgy]
					if rot in rotation_to_idx and
					rot in LEAVE_ALLOWED.get(pgy, set())
				]

			x[r, b] = model.NewIntVarFromDomain(
				cp_model.Domain.FromValues(eligible), f'x_{r}_{b}'
			)


# -------------------------
# Constraints
# -------------------------

# Create indicator variables y[r, b, rot]
y = {}
for r in range(n):
	for b in range(BLOCKS):
		for rot in PER_BLOCK_MIN:
			y[r, b, rot] = model.NewBoolVar(f'y_{r}_{b}_{rot}')
			rot_idx = rotation_to_idx[rot]
			model.Add(x[r, b] == rot_idx).OnlyEnforceIf(y[r, b, rot])
			model.Add(x[r, b] != rot_idx).OnlyEnforceIf(
				y[r, b, rot].Not()
			)

# Use integer weights: 2 (full), 1 (half)
half_leave_weights = {
	(r, b): 1 if (b + 1) in leave_dict[residents[r]]["half"] else 2
	for r in range(n)
	for b in range(BLOCKS)
}

# Enforce scaled minimum coverage
for b in range(BLOCKS):
	for rot in PER_BLOCK_MIN:
		model.Add(
			sum(
				half_leave_weights[(r, b)] * y[r, b, rot]
				for r in range(n)
			) >= 2 * PER_BLOCK_MIN[rot]
		)


# Graduation Requirements
for r in range(n):
	res_id = residents[r]
	pgy = pgys[r]
	leave_blocks = leave_dict[res_id]["full"]

	for rotation_group, required_counts in GRAD_REQ[pgy].items():
		# Skip this specific R3 exemption if full leave taken
		if pgy == "R3" and rotation_group == (
			"Cardiology", "ED", "Medical Consultation"
		) and leave_blocks:
			continue

		match_vars = []
		for b in range(BLOCKS):
			match = model.NewBoolVar(f'grad_{r}_{b}_{rotation_group}')
			model.AddAllowedAssignments(
				[x[r, b]],
				[[rotation_to_idx[rot]]                        # allowed
				 for rot in rotation_group if rot in rotation_to_idx]
			).OnlyEnforceIf(match)
			model.AddForbiddenAssignments(
				[x[r, b]],
				[[rotation_to_idx[rot]]                        # forbidden
				 for rot in ROTATIONS
				 if rot not in rotation_group and rot in rotation_to_idx]
			).OnlyEnforceIf(match)
			match_vars.append(match)

		model.Add(sum(match_vars) >= min(required_counts))
		model.Add(sum(match_vars) <= max(required_counts))


group_defs = {
	"FLOATER": {"Nephrology", "Endocrine"},
	"NEURO": {"Neurology"},
	"2NDONCALL": {"GI", "Rheumatology", "Pulmonology"}
}

z = {}
for r in range(n):
	for b in range(BLOCKS):
		for group_name, rotations in group_defs.items():
			z[r, b, group_name] = model.NewBoolVar(f'z_{r}_{b}_{group_name}')

			rot_ids = [
				rotation_to_idx[rot]
				for rot in rotations if rot in rotation_to_idx
			]
			if not rot_ids:
				model.Add(z[r, b, group_name] == 0)
				continue

			group_matches = []
			for rot_id in rot_ids:
				match = model.NewBoolVar(f'is_{residents[r]}_{b}_r{rot_id}')
				model.Add(x[r, b] == rot_id).OnlyEnforceIf(match)
				model.Add(x[r, b] != rot_id).OnlyEnforceIf(match.Not())
				group_matches.append(match)

			model.AddBoolOr(group_matches).OnlyEnforceIf(z[r, b, group_name])
			model.AddBoolAnd([m.Not() for m in group_matches])\
				.OnlyEnforceIf(z[r, b, group_name].Not())

for b in range(BLOCKS):
	# model.Add(
	# 	sum(y[r, b, "Senior Rotation"] for r in range(n)) == 10
	# )

	# model.Add(
	# 	sum(y[r, b, "Registrar Rotation"] for r in range(n)) == 20
	# )

	model.Add(
		sum(half_leave_weights[(r, b)] *
			y[r, b, "Senior Rotation"] for r in range(n)) == 2 * 10
	)
	model.Add(
		sum(half_leave_weights[(r, b)] *
			y[r, b, "Registrar Rotation"] for r in range(n)) == 2 * 20
	)

	model.Add(
		sum(z[r, b, "FLOATER"] for r in range(n)) >= 8
	)

	model.Add(
		sum(z[r, b, "NEURO"] for r in range(n)) >= 4
	)

	model.Add(
		sum(half_leave_weights[(r, b)] * z[r, b, "2NDONCALL"]
			for r in range(n)) >= 20
	)

for b in range(3, BLOCKS):
	# model.Add(
	# 	sum(y[r, b, "Senior Rotation"] for r in range(n)) == 10
	# )

	# model.Add(
	# 	sum(y[r, b, "Registrar Rotation"] for r in range(n)) == 20
	# )

	model.Add(
		sum(half_leave_weights[(r, b)] *
			y[r, b, "Medical Teams"] for r in range(n)) == 2 * 20
	)


# -------------------------
# Block-1 PGY rules
# -------------------------
med_teams_idx = rotation_to_idx["Medical Teams"]
senior_idx = rotation_to_idx["Senior Rotation"]

for r in range(n):
	if pgys[r] == "R1":
		model.Add(x[r, 0] == med_teams_idx)
	elif pgys[r] == "R2":
		model.Add(x[r, 0] != senior_idx)

# -------------------------
# Consecutive-Medical-Teams guard (R1)
# -------------------------
for r in range(n):
	if pgys[r] == "R1":
		for start in range(BLOCKS - 5):       # window size = 6
			model.Add(
				sum(y[r, b, "Medical Teams"] for b in range(start, start + 6))
				<= 5
			)


In [ ]:

# TODO: First block for R1 must be medical teams.
# 1. R2 cannot be senior in the first block.
# 2. R1 first block must be Medical Teams.
# 3. There should be a minimum number of residents available per half (add half block constraint).
# 4. Add maximum number of residents for certain rotations, but for a specific number of blocks this constraint do not need to be satisified.
# 5. Add preferences (e.g. 1 Senior in Cardiology)
# 6. Add summary per block (mention floaters/ 2nd on-call etc.).
# 7. R1 cannot do more than 5 Medical Teams Rotations in a row.
# Color Code + Predetrimned blocks as options in the input file.
